In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

 ## Połączenie z bazą danych


In [2]:
username = 'postgres'
password = 'postgres'

host = 'localhost'
database = 'airline'
port = 5432

In [3]:
url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

In [4]:
def read_sql_table(table_name):
    msg = f"Pobieram dane z tabeli: {table_name}"
    print(msg)

    df = pd.read_sql(table_name, engine)
    return df

In [5]:
flight_df = pd.read_csv(
    '../data/processed/flight_df_02.csv',
    sep=';')

 # Wzbogacenie o `airport_list`


In [6]:
airport_list_df = read_sql_table('airport_list')

Pobieram dane z tabeli: airport_list


In [7]:
test = airport_list_df.drop_duplicates(subset='origin_airport_id').shape[0]
test == airport_list_df.shape[0]

True

In [8]:
flight_df = pd.merge(
    left=flight_df,
    right=airport_list_df[['origin_airport_id', 'origin_city_name']],
    how='left',
    on=['origin_airport_id']
)

In [9]:
tmp_airport_list_df = airport_list_df.rename(
    columns={'origin_city_name': 'destination_city_name'})

flight_df = pd.merge(
    left=flight_df,
    right=tmp_airport_list_df[['origin_airport_id', 'destination_city_name']],
    how='left',
    left_on=['dest_airport_id'],
    right_on=['origin_airport_id']
)

flight_df = (flight_df
             .drop(columns='origin_airport_id_y')
             .rename(columns={'origin_airport_id_x': 'origin_airport_id'})
             )

### Sprawdzenie


In [10]:
assert 'origin_city_name' in flight_df.columns, 'Brak kolumny `origin_city_name` w ramce flight_df'
assert 'destination_city_name' in flight_df.columns, 'Brak kolumny `destination_city_name` w ramce flight_df'

flight_df_expected_rows_amount = 1057391
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'

 ## Analiza według lotnisk oraz tras


In [11]:
top_airports_origin_df = (flight_df
                          .groupby(by=['origin_airport_id']).size()
                          )
top_airports_origin_df


origin_airport_id
10299      6966
10397    123162
10721     55989
10821     12102
11057     12047
11193      5385
11259      6057
11278     29744
11292     64525
11298     39511
11433      6591
11618     24230
11697      8612
12173     30565
12191      6052
12266     11232
12402      5091
12478     18077
12758      7792
12889     38456
12892     87849
12953     57708
12982      6933
13204     26997
13303     12605
13487     13033
13495      5283
13830     10718
13930    105437
14100     19595
14107     36517
14492      5275
14679     24912
14747     50465
14771     50124
14831      6309
14869     12428
14893      5852
15304      7165
dtype: int64

In [12]:
top_airports_destination_df = (flight_df
                               .groupby(by=['dest_airport_id']).size()
                               )
top_airports_destination_df


dest_airport_id
10299      7199
10397    122945
10693      5185
10721     56057
10821     12163
11057     12016
11193      5374
11259      6052
11278     29759
11292     64602
11298     39488
11433      6619
11618     24227
11697      8628
12173     30534
12191      6057
12266     11225
12402      5097
12478     18083
12758      7803
12889     38494
12892     87776
12953     57686
12982      6940
13204     21760
13303     12613
13487     13055
13495      5331
13830     10725
13930    100333
14100     19573
14107     36499
14492      5293
14679     24926
14747     50230
14771     49999
14831      6328
14843      5206
14869     12504
14893      5850
15304      7157
dtype: int64

 ### Sprawdzenie dla `top_airport_origin`

In [13]:
top_airports_origin_head = (top_airports_origin_df
                            .sort_values(ascending=False)
                            .head()
                            .to_list()
                            )
top_airports_origin_head = tuple(top_airports_origin_head)
top_airports_origin_head_expected = (123162, 105437, 87849, 64525, 57708)

assert top_airports_origin_head == top_airports_origin_head_expected, f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_origin_head_expected} otrzymano: {top_airports_origin_head}"


### Sprawdzenie dla `top_airport_destination`

In [14]:
top_airports_destination_head = (top_airports_destination_df
                                 .sort_values(ascending=False)
                                 .head()
                                 .to_list()
                                 )
top_airports_destination_head = tuple(top_airports_destination_head)
top_airports_destination_head_expected = (122945, 100333, 87776, 64602, 57686)

assert top_airports_destination_head == top_airports_destination_head_expected, f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_destination_head_expected} otrzymano: {top_airports_destination_head}"


 # Wzbogacenie o dane pogodowe
 

In [15]:
airport_weather_df = read_sql_table('airport_weather')

Pobieram dane z tabeli: airport_weather


In [16]:
aiport_weather_df_columns = ['station', 'name',
                             'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']
airport_weather_df = airport_weather_df[aiport_weather_df_columns]


In [17]:
airport_weather_df = pd.merge(
    left=airport_weather_df,
    right=airport_list_df[['origin_airport_id', 'name']],
    how='inner',
    on='name'
)

 ### Sprawdzenie


In [18]:
airport_weather_df_expected_shape = (43394, 9)
airport_weather_df_shape = airport_weather_df.shape

assert airport_weather_df_expected_shape == airport_weather_df_shape, \
  f'Nieodpowiedni wymiar ramki airport_weather_df, oczekiwano (wierszy, kolumn): {airport_weather_df_expected_shape}'


 ## Połączenie `airport_weather_df` oraz `flight_df`


In [19]:
airport_weather_df.sample(100).head(100)

station                                               name  \
22031  USW00013994      ST LOUIS LAMBERT INTERNATIONAL AIRPORT, MO US   
36667  USW00023185                                RENO AIRPORT, NV US   
27851  USW00053922  FAYETTEVILLE SPRINGDALE NW AR REGL AIRPORT, AR US   
2852   USW00004804      COLUMBUS OHIO STATE UNIVERSITY AIRPORT, OH US   
3687   USW00003017                DENVER INTERNATIONAL AIRPORT, CO US   
...            ...                                                ...   
1439   USW00013881                   CHARLOTTE DOUGLAS AIRPORT, NC US   
37299  USW00012917          PORT ARTHUR SE TX REGIONAL AIRPORT, TX US   
20233  USW00024229              PORTLAND INTERNATIONAL AIRPORT, OR US   
21812  USW00013994      ST LOUIS LAMBERT INTERNATIONAL AIRPORT, MO US   
2361   USW00014820     CLEVELAND HOPKINS INTERNATIONAL AIRPORT, OH US   

             date  prcp  snow  snwd  tmax   awnd  origin_airport_id  
22031  2020-02-25  0.17   0.0   0.0  43.0  12.08              15016  
36667  2019-01-23  0.00   0.0   0.0  58.0   3.36              14570  
27851  2019-11-09  0.00   0.0   0.0  62.0   8.28              15919  
2852   2019-04-27  0.09   NaN   NaN  58.0   8.28              11066  
3687   2019-05-11  0.02   0.0   0.0  62.0  10.96              11292  
...           ...   ...   ...   ...   ...    ...                ...  
1439   2019-03-12  0.00   0.0   0.0  64.0   6.04              11057  
37299  2019-07-18  0.01   NaN   NaN  93.0   8.28              13487  
20233  2020-03-22  0.00   0.0   0.0  63.0   3.80              14057  
21812  2019-07-21  0.00   0.0   0.0  91.0   7.61              15016  
2361   2019-03-23  0.00   0.0   0.0  46.0   8.05              11042  

[100 rows x 9 columns]

In [20]:
airport_weather_df['date'] = pd.to_datetime(
    airport_weather_df['date'],
    format='%Y-%m-%d'
)

In [21]:
flight_df['date'] = (flight_df['year'].astype(str)
                     + '-' + flight_df['month'].astype(str)
                     + '-' + flight_df['day_of_month'].astype(str)
                     )
flight_df['date'] = pd.to_datetime(flight_df['date'])


In [22]:
flight_df = pd.merge(
    left=flight_df,
    right=airport_weather_df,
    how='left',
    on=['origin_airport_id', 'date']
)

 ### Sprawdzenie


In [23]:
flight_df_expected_rows_amount = 1057391
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'


 ## Analiza dla kolumny `tmax`

In [24]:
flight_df['tmax'].describe().round(2)

count    1051956.00
mean          70.33
std           17.97
min          -10.00
25%           59.00
50%           72.00
75%           84.00
max          115.00
Name: tmax, dtype: float64

In [25]:
scope = flight_df
percentiles = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
scope['tmax'].describe(percentiles=percentiles).round(2)

count    1051956.00
mean          70.33
std           17.97
min          -10.00
5%            38.00
10%           45.00
20%           55.00
30%           62.00
40%           67.00
50%           72.00
60%           76.00
70%           81.00
80%           87.00
90%           92.00
95%           95.00
99%          106.00
max          115.00
Name: tmax, dtype: float64

In [26]:
delay_by_tmax_df = scope.pivot_table(
    index=pd.cut(scope['tmax'], range(-10, 106, 10)),
    values='dep_delay',
    aggfunc=['mean', 'count']
)
delay_by_tmax_df

mean     count
           dep_delay dep_delay
tmax                          
(0, 10]    42.385356      1038
(10, 20]   26.774660      4562
(20, 30]   21.080204     15710
(30, 40]   21.697595     48316
(40, 50]   14.204016     86851
(50, 60]   13.267904    131629
(60, 70]   12.776241    202235
(70, 80]   13.883600    227508
(80, 90]   14.881341    200583
(90, 100]  17.028844    113821
(-10, 0]         NaN         0

In [27]:
flight_df.loc[flight_df['tmax'] < 0 ]['tmax'].unique()

array([-10.])

In [28]:
flight_df = flight_df.loc[flight_df['tmax'] > 0]

In [29]:
plt.bar(
    x=delay_by_tmax_df.index.astype(str),
    height=delay_by_tmax_df[('mean', 'dep_delay')],
)

<BarContainer object of 11 artists>